# Read Libs

In [2]:
import pandas as pd
import datetime as dt
import pdb
import numpy as np
import requests as req
import selenium
import os



# display and output settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

display(HTML('''
<style>
    .container { width:85% !important; }
</style>
'''))

pd.set_option('display.min_rows', 100)